### Ingest pit_stops.json file

In [3]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

dbutils.widgets.text("p_file_date", "2021-03-28")
v_file_date = dbutils.widgets.get("p_file_date")

In [5]:
from formula1.formula1_constants import *
from formula1.formula1_utils import *

##### Step 1 - Read the JSON file using the spark dataframe reader API

In [6]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import lit

pit_stops_schema = StructType(
    fields=[
        StructField("raceId", IntegerType(), False),
        StructField("driverId", IntegerType(), True),
        StructField("stop", StringType(), True),
        StructField("lap", IntegerType(), True),
        StructField("time", StringType(), True),
        StructField("duration", StringType(), True),
        StructField("milliseconds", IntegerType(), True),
    ]
)

In [7]:
pit_stops_df = (
    spark.read.schema(pit_stops_schema)
    .option("multiLine", True)
    .json(f"{landing_folder_path}/{v_file_date}/pit_stops.json")
)

##### Step 2 - Add ingestion meta data

In [8]:
pit_stops_with_metadata_df = (
    add_ingestion_date(pit_stops_df)
    .withColumn("data_source", lit(v_data_source))
    .withColumn("file_date", lit(v_file_date))
)

##### Step 3 - Write to output to bronze container

In [9]:
pit_stops_with_metadata_df.write.mode("overwrite").format("delta").saveAsTable(
    "f1_bronze.pit_stops"
)